In [1]:
%xmode verbose
import torch
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import layoutparser as lp

from pdf2image import convert_from_path
from PIL import Image

#For Detectron2
import torch
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import layoutparser as lp

import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import cv2

import os

os.chdir('..')

path= ''
prediction_score_threshold = 0.7
class_labels = ['text', 'title', 'list', 'table', 'figure']

# Set up Detectron2 config
cfg = get_cfg()
#The following line can be used instead of the one after it, however it causes warnings. (Find out why?)
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a threshold for predictions
cfg.MODEL.WEIGHTS = path+ "model_final.pth" 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

cfg.MODEL.DEVICE = "cpu"



predictor = DefaultPredictor(cfg)

Exception reporting mode: Verbose


In [2]:
def pdf_to_image(path):
    """
    Converts a PDF Document into a list of images     
    Args:
    pdf_path (String): Path to the pdf file for information extraction.

    Returns:
    A list of images (each page is an image) in the format [[1,image1],[2,image2]]
    """
    
    


    # Convert PDF to images
    images = convert_from_path(path)
    return enumerate(images)

In [3]:
def pdf_to_elements_detectron(pdf_path, display= False) :
    """
    Extracts elements from a pdf file by utilizing detectron2
    
    Args:
    pdf_path (String): Path to the pdf file for information extraction.
    
    Returns:
    An array of extracted texts
    """
    
    path= ''
    prediction_score_threshold = 0.7
    class_labels = ['text', 'title', 'list', 'table', 'figure']

    # Set up Detectron2 config
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a threshold for predictions
    cfg.MODEL.WEIGHTS = path+ "model_final.pth" # e.g., PubLayNet pre-trained model
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

    cfg.MODEL.DEVICE = "cpu"

    # Initialize the OCR library (ensure tesseract is installed and in PATH)
    #pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'  # Update if necessary

    # Initialize an empty list to store the content of each detected object
    detected_texts = []

    predictor = DefaultPredictor(cfg)
    
    images= pdf_to_image(pdf_path)
    for i, current_image in images:
        #img = np.array(Image.open(current_image))
        img = np.array(current_image)

        # Perform page object detection
        outputs = predictor(img)

        # Debug outputs
        instances = outputs["instances"].to("cpu")
        pred_boxes = instances.pred_boxes
        scores = instances.scores
        pred_classes = instances.pred_classes

        # Loop through each detected object
        for i in range(0, len(pred_boxes)):
            box = pred_boxes[i].tensor.numpy()[0]
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])

            # Crop the image to the bounding box
            cropped_img = img[y1:y2, x1:x2]

            # Perform OCR on the cropped image
            text = pytesseract.image_to_string(cropped_img, output_type=Output.STRING)

            # Append the extracted text to the list

            label_key = int(pred_classes[i].numpy())
            label = class_labels[label_key]
            detected_texts.append([label,text.strip()])  # Store each text as a single-element list


            #print(f"Detected {label}: {text.strip()}")

        if display:

            for i in range(0, len(pred_boxes)):
                box = pred_boxes[i].tensor.numpy()[0]
                score = round(float(scores[i].numpy()), 4)
                label_key = int(pred_classes[i].numpy())
                label = class_labels[label_key]
                x = int(box[0])
                y = int(box[1])
                w = int(box[2] - box[0])
                h = int(box[3] - box[1])

                print('Detected object of label=' + str(label) + ' with score=' + str(score) + ' and in box={x=' + str(x) + ', y=' + str(y) + ', w=' + str(w) + ', h=' + str(h) + '}')
            # Draw the bounding boxes
            for i in range(len(pred_boxes)):
                box = pred_boxes[i].tensor.numpy()[0]
                score = round(float(scores[i].numpy()), 4)
                label_key = int(pred_classes[i].numpy())
                label = class_labels[label_key]

                x = int(box[0])
                y = int(box[1])
                w = int(box[2] - box[0])
                h = int(box[3] - box[1])

                # Draw the rectangle around each object
                cv2.rectangle(img, (x, y), (x + w, y + h), color=(0, 255, 0), thickness=2)

                # Put label and score text above the bounding box
                cv2.putText(img, f"{label}: {score}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Convert the image from BGR to RGB (for matplotlib display)
            image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Display the image with bounding boxes
            plt.figure(figsize=(10, 10))
            plt.imshow(image_rgb)
            plt.axis("off")  # Hide the axis
            plt.show()
    
    return detected_texts

In [4]:
pdf_path= path+'biology_paper.pdf'

In [5]:
%xmode verbose

pdf_to_elements_detectron(pdf_path, display= False) #Change display to true to vizualize the boxes

Exception reporting mode: Verbose


The checkpoint state_dict contains keys that are not used by the model:
  roi_heads.mask_head.mask_fcn1.{bias, weight}
  roi_heads.mask_head.mask_fcn2.{bias, weight}
  roi_heads.mask_head.mask_fcn3.{bias, weight}
  roi_heads.mask_head.mask_fcn4.{bias, weight}
  roi_heads.mask_head.deconv.{bias, weight}
  roi_heads.mask_head.predictor.{bias, weight}
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[['title',
  'A new jewelflower (Streptanthus, Brassicaceae) from Marin County\nCalifornia, USA.'],
 ['text', 'Article in Madrofio - June 2019'],
 ['text', '4 authors, includin;'],
 ['title',
  'A NEW JEWELFLOWER (STREPTANTHUS,\nBRASSICACEAE) FROM MARIN COUNTY, CALIFORNIA,\nUSA'],
 ['text',
  'BioOne sees sustainable scholarly publishing as an inherently collaborative enterprise connecting authors, nonprofit\npublishers, academic institutions, research libraries, and research funders in the common goal of maximizing access to critical\nresearch.'],
 ['title',
  'Source: Madrono, 66(1) : 24-29\nPublished By: California Botanical Society\nURL: https://doi.org/10.3120/0024-9637-66.1.'],
 ['text',
  'Your use of this PDF, the BioOne Complete website, and all posted and associated content indicate:\nyour acceptance of BioOne’s Terms of Use, available at www.bioone.org/terms-of-use.'],
 ['text',
  'Usage of BioOne Complete content is strictly limited to personal, educational, and non-commerc

In [6]:
from unstructured.partition.auto import partition

def pdf_to_elements_yolox(pdf_path) :
    """
    Extracts elements from a pdf file by utilizing yolox without the need of an API
    
    Args:
    pdf_path (String): Path to the pdf file for information extraction.
    
    Returns:
    A list of Unstructured Elements
    """
    
    try:
        elements = partition(filename=pdf_path,strategy='hi_res',skip_infer_table_types=[])
        i = -1
        for el in elements:
            i+=1
            if(el.category =='Table'):
                elements[i].text=elements[i].metadata.text_as_html
            if(el.category =='Header'):
                elements[i].category="Title"
        return  elements
    except Exception as e:
        print(e)
        return []
        

In [7]:
from typing import Dict, List, Any
from unstructured.documents.elements import Element
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from pdfminer.layout import LTTextBoxHorizontal
from unstructured.documents.elements import Element, ElementMetadata, CoordinateSystem

def _extract_text_with_layout(pdf_path):
    """
    Extracts text and layout information from a PDF, capturing text along with its position and font size.

    Args:
    pdf_path (str): Path to the PDF file for extracting text and layout information.

    Returns:
    List[Dict[str, Any]]: A list of dictionaries where each dictionary contains text, bounding box (bbox), and font size of the text element.
    """
    text_with_layout = []
    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextBoxHorizontal):
                for text_line in element:
                    text_with_layout.append({
                        'text': text_line.get_text(),
                        'bbox': text_line.bbox,  # (x0, y0, x1, y1)
                        'font_size': text_line._objs[0].size if text_line._objs else None
                    })
    return text_with_layout

def _detect_titles(text_with_layout, title_font_size_threshold=14):
    """
    Detects potential titles in the extracted text based on font size and positioning.

    Args:
    text_with_layout (List[Dict[str, Any]]): List of text elements with layout information (text, bbox, font size).
    title_font_size_threshold (int, optional): The minimum font size considered to identify a title. Default is 14.

    Returns:
    List[Dict[str, Any]]: A list of dictionaries containing title text, bounding box, and font size.
    """
    titles = []
    for item in text_with_layout:
        text = item['text'].strip()
        if text:
            font_size = item['font_size']
            if font_size and font_size >= title_font_size_threshold:
                titles.append({
                    'text': text,
                    'bbox': item['bbox'],
                    'font_size': font_size
                })
    return titles

def _is_string_in_titles(search_string: str, titles: List[Dict[str, Any]]) -> bool:
    """
    Checks if a specific string is present in the list of detected titles.

    Args:
    search_string (str): The string to search for within the titles.
    titles (List[Dict[str, Any]]): List of title elements (text, bbox, font size) identified by `_detect_titles`.

    Returns:
    bool: True if the search string is found within any of the titles, otherwise False.
    """
    search_string = search_string.lower()
    for title in titles:
        if search_string in title['text'].lower():
            return True
    return False


def pdf_to_elements_pdfminer(file_path: str) -> List[Element]:
    """
    Extracts text elements from a PDF and returns them as a list of `Element` instances, 
    with metadata such as coordinates, page number, and distinguishing between titles and narrative text.

    Args:
    file_path (str): Path to the PDF file.

    Returns:
    List[Unstructured Element]: A list of `Element` objects representing the text elements in the PDF, with associated metadata such as coordinates and category (title or narrative text).
    """
    extracted_elements = []

    titles = _detect_titles(_extract_text_with_layout(file_path))
    
    try:
        for page_number, page_layout in enumerate(extract_pages(file_path), start=1):
            for page_element in page_layout:
                if isinstance(page_element, LTTextContainer):
                    text = page_element.get_text().strip()
                    if text:
                        x0, y0, x1, y1 = page_element.bbox
                        element_type = "Title" if _is_string_in_titles(search_string=text, titles=titles) else "'NarrativeText'"
                        
                        coordinate_system = CoordinateSystem(
                            width=x1 - x0,
                            height=y1 - y0
                        )
                        
                        metadata = ElementMetadata(
                            filename=file_path,
                            page_number=page_number,
                            coordinates=coordinate_system,
                            languages=['en']
                        )
                        
                        unstructured_element = Element(
                            element_id=None,
                            coordinates=((x0, y0), (x1, y1)),
                            coordinate_system=coordinate_system,
                            metadata=metadata,
                            detection_origin=element_type
                        )

                        unstructured_element.category = element_type
                        unstructured_element.text = text
                        extracted_elements.append(unstructured_element)

    except Exception as e:
        raise

    return extracted_elements

In [ ]:
yolo_elements = pdf_to_elements_yolox(pdf_path)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

2024-10-15 10:14:52.231711: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 10:14:52.240263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 10:14:52.249152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 10:14:52.251700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 10:14:52.260553: I tensorflow/core/platform/cpu_feature_guar

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
yolo_elements[0].category

In [ ]:
yolo_elements[0].text

In [ ]:
elements = pdf_to_elements_pdfminer(pdf_path)

In [ ]:
elements[0].category


In [ ]:
elements[0].text